In [4]:
# Some basic imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import warnings
import ast
import re

In [ ]:
# Load the general data:
success_movies_path = 'data/processed_data/successful_movies.csv'
ethnicity_path = 'data/processed_data/diversity_final.csv'

success_movies_df = pd.read_csv(success_movies_path, sep=',')
ethnicity_df = pd.read_csv(ethnicity_path, sep=',')


## Assessing Correlation between Diversity and Success

- Correlation Analysis: Evaluate the dependency between diversity and film success using: Pearson correlation (stats.pearson) for linear associations.

- Spearman correlation (stats.spearman) 

- Regression Analysis: Investigate the association type (e.g., weak, negative, positive) through linear regression.